In [1]:
import cv2
import numpy as np

In [2]:
def pre_processing(img):
    
    kernel = np.ones((5,5), np.uint8)
    
    grayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurImg = cv2.GaussianBlur(grayImg, (5,5), 1)
    cannyImg = cv2.Canny(img, 200, 200)
    dilatedImg = cv2.dilate(cannyImg, kernel, iterations=2) 
    resultImg = cv2.erode(dilatedImg, kernel, iterations=1)
    
    return resultImg

In [3]:
def get_contour(img):
    
    biggest = np.array([])
    maxArea = 0
    
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area>5000:            
            #cv2.drawContours(contourFrame, cnt, -1, (255,0,0), 2)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            
            if area>maxArea and (len(approx) == 4):
                biggest = approx
                maxArea = area
                
    cv2.drawContours(contourFrame, biggest, -1, (255,0,0), 20)
    
    return biggest

In [4]:
def get_warp(img, biggest):
    
    cntPt = reorder(biggest)
    width, height = 340, 450
    pts1 = np.float32(cntPt)
    pts2 = np.float32([[0,0], [width,0], [0,height], [width,height]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    trsfrmImg = cv2.warpPerspective(img, matrix, (width,height))
    
    outputImg = trsfrmImg[10:trsfrmImg.shape[0]-10, 10:trsfrmImg.shape[1]-10]
    outputImg = cv2.resize(outputImg, (width, height))
    
    return outputImg

In [5]:
def reorder(pts):
    pts = pts.reshape((4,2))
    newPts = np.zeros((4,1,2), np.int32)
    
    sum_ = pts.sum(1)
    
    newPts[0] = pts[np.argmin(sum_)] #Smallest point
    newPts[3] = pts[np.argmax(sum_)] #biggest point
    
    diff = np.diff(pts, axis=1)
    
    newPts[1] = pts[np.argmin(diff)]
    newPts[2] = pts[np.argmax(diff)]
    
    return newPts

In [6]:
frameWid = 345
frameHig = 500

camFlag = False
cap = cv2.VideoCapture(0)
cap.set(3, frameWid)
cap.set(4, frameHig)
cap.set(10,150)

while(True):
    if camFlag:
        ret, frame = cap.read()
    else: 
        frame = cv2.imread('Images/document.jpeg')
    
    frame = cv2.resize(frame, (frameWid,frameHig))
    contourFrame = frame.copy()
    
    threshImg = pre_processing(frame)
    biggest = get_contour(threshImg)
    
    if biggest.size != 0:
        warp = get_warp(frame, biggest)
        cv2.imshow('Final Image',warp)
    
    cv2.imshow('Image',frame)
    cv2.imshow('Contour Image',contourFrame)
    
    
    if cv2.waitKey(1) == ord('q'):
        break

if camFlag: cap.release()
    
cv2.destroyAllWindows()